In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc

2020-12-22 11:03:52,021 - NumExpr defaulting to 8 threads.


In [2]:
path = '../raw_data/'

In [3]:
at_raw = pd.read_csv(path+'active_totes_20201123.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-2511.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv('../raw_data/SCS alerts Nov.csv')

In [4]:
# HOTFIX - remove odd day
at_raw = at_raw[~at_raw['DAY'].isin([9])]

In [5]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2138157551680) was called 1 times


In [6]:
at = feat.pre_process_AT(at_raw)

Running: Function "pre_process_AT" (id=2138157554560) was called 1 times


In [7]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2138157555424) was called 1 times


In [8]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2138157554128) was called 1 times
Running: Function "load_module_lookup" (id=2138157552976) was called 1 times
Running: Function "load_ID_lookup" (id=2138157553696) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [9]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0Minutes


In [10]:
mod = [1]
limit = 250

In [11]:
fa_sel_1 = feat.get_data_faults(fa_floor, modules = mod)

most_common = list(((fa_sel_1['Asset Code'].value_counts() > limit)[fa_sel_1['Asset Code'].value_counts() > limit]).index)
fa_sel = fa_sel_1[fa_sel_1['Asset Code'].isin(most_common)]

In [12]:
#fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Blue']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel, fault_agg_level= 'Asset Code', agg_type = 'count')

In [13]:
av_sel, at_sel = feat.av_at_select(av, at, remove_high_AT = True, availability_select_options = {"Module" : mod})

av_agg = feat.aggregate_availability(av_sel, agg_level = 'Module')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'Module')

Running: Function "aggregate_availability" (id=2138157554704) was called 1 times


In [14]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg, agg_level = 'None')

In [15]:
# df = (pd.merge(at_agg.reset_index(), av_agg.reset_index(), on=['timestamp', 'Module'], how='left')
#  .merge(fa_agg, on=['timestamp'], how='left')).drop(['Module', 'Blue Tote Loss', 'Grey Tote Loss'], axis=1)
# df = df.fillna(0)

In [16]:
X,y = md.gen_feat_var(df)
X = X.drop('Module', axis=1)
# Flip to availability
y=1-y

In [17]:
X_train, X_test, y_train, y_test = md.split(X,y)

#RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,num_trees=100, criterion = 'mse', max_depth=None, dtree=False)

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

#XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test)

maxf = None

# X_sel_rf = md.select_features(X, y, RF_mdl, thres = -np.inf, max_feat = maxf)

#X_sel_lm = md.select_features(X, y, Linear_mdl, selection_options={'thres': 'mean', 'max_feat': None})

#X_sel_XGB = md.select_features(X, y, XGB_mdl, selection_options={'thres': 'mean', 'max_feat': None})

#X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,test_size=0.3,random_state=101)

#Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)


cv_R2 = md.cross_validate_r2(Linear_mdl, X, y, n_folds = 5, shuffle = True, random_state = 101)

#cv_R2 = md.cross_validate_r2(RF_mdl, X_sel_rf, y, n_folds = 10, shuffle = True, random_state = 101)

#cv_R2 = md.cross_validate_r2(XGB_mdl, X, y, n_folds = 10, shuffle = True, random_state = 101)

#md.stats_model(Linear_mdl,X_test,y_test)


Cross Validation Scores LinearRegression(): 
 
        R2 Scores
1       0.518717
2       0.149442
3       0.409224
4       0.275604
5       0.588771
Mean    0.388352
STD     0.159617


In [19]:
model = sm.OLS(y,X)
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:               Downtime   R-squared (uncentered):                   0.937
Model:                            OLS   Adj. R-squared (uncentered):              0.931
Method:                 Least Squares   F-statistic:                              152.1
Date:                Tue, 22 Dec 2020   Prob (F-statistic):                    1.17e-72
Time:                        11:05:12   Log-Likelihood:                          81.412
No. Observations:                 145   AIC:                                     -136.8
Df Residuals:                     132   BIC:                                     -98.13
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

In [20]:
keep_features = results.pvalues[results.pvalues < 0.10].index

In [22]:
model = sm.OLS(y,X[keep_features])
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:               Downtime   R-squared (uncentered):                   0.936
Model:                            OLS   Adj. R-squared (uncentered):              0.933
Method:                 Least Squares   F-statistic:                              337.6
Date:                Tue, 22 Dec 2020   Prob (F-statistic):                    3.01e-80
Time:                        11:05:24   Log-Likelihood:                          79.514
No. Observations:                 145   AIC:                                     -147.0
Df Residuals:                     139   BIC:                                     -129.2
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

In [23]:
negs = results.params[results.params < 0].index
negs

Index(['C1603BCL014'], dtype='object')

In [24]:
df['timestamp'].min()

Timestamp('2020-11-17 09:00:00')

In [30]:
list(negs.values)

['C1603BCL014']

In [31]:
fa[fa['Asset Code'].isin(list(negs.values))]

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
80140,130233.0,C16: C1603BCL014 014B54 sensor fault,2020-11-02 15:33:00,C16,P03,1.0,3428,C1603BCL014,Blue,Outside,C1603,False,16.0,0.0,NaN,Fault
81391,130234.0,C16: C1603BCL014 014B55 sensor fault,2020-11-13 02:48:00,C16,P03,1.0,3428,C1603BCL014,Blue,Outside,C1603,False,16.0,0.0,NaN,Fault
81313,130234.0,C16: C1603BCL014 014B55 sensor fault,2020-11-13 23:22:00,C16,P03,1.0,3428,C1603BCL014,Blue,Outside,C1603,False,16.0,0.0,NaN,Fault
81335,130231.0,C16: C1603BCL014 014B52 sensor fault,2020-11-13 18:14:00,C16,P03,1.0,3428,C1603BCL014,Blue,Outside,C1603,False,16.0,0.0,NaN,Fault
81857,130234.0,C16: C1603BCL014 014B55 sensor fault,2020-11-22 00:45:00,C16,P03,1.0,3428,C1603BCL014,Blue,Outside,C1603,False,16.0,0.0,NaN,Fault
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80369,130233.0,C16: C1603BCL014 014B54 sensor fault,2020-11-04 03:57:00,C16,P03,351.0,3428,C1603BCL014,Blue,Outside,C1603,False,16.0,0.0,NaN,Fault
81363,130233.0,C16: C1603BCL014 014B54 sensor fault,2020-11-13 14:40:00,C16,P03,381.0,3428,C1603BCL014,Blue,Outside,C1603,False,16.0,0.0,NaN,Fault
80578,130233.0,C16: C1603BCL014 014B54 sensor fault,2020-11-07 09:37:00,C16,P03,522.0,3428,C1603BCL014,Blue,Outside,C1603,False,16.0,0.0,NaN,Fault
80687,130233.0,C16: C1603BCL014 014B54 sensor fault,2020-11-08 11:43:00,C16,P03,527.0,3428,C1603BCL014,Blue,Outside,C1603,False,16.0,0.0,NaN,Fault


In [25]:
fa[fa['Asset Code'].isin([negs]) & (fa['timestamp'] >= df['timestamp'].min())]#['Fault ID'].value_counts()

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
